In [ ]:
import pvlib
import pandas as pd

In [ ]:
location_latitude=19.076
location_longitude=72.8777

# Q1

# PVlib

# calculate solar zenith solar azimuth

In [ ]:
time_hour = pd.date_range(start ='2015-01-01 01:00', freq ='H',  periods =8760, tz ='Asia/Calcutta') # date_range bcpz pythongave error for Datetime

In [ ]:
solarpos=pvlib.solarposition.get_solarposition(time=time_hour,latitude=location_latitude, longitude=location_longitude, altitude=None, pressure=None, method='nrel_numpy')

In [ ]:
location_tilt=location_latitude
location_azimuth=180 #south facing

# calculate aoi

In [ ]:
irr_aoi=pvlib.irradiance.aoi(surface_tilt=location_tilt, surface_azimuth=location_azimuth, solar_zenith=solarpos.zenith, solar_azimuth=solarpos.azimuth)

In [ ]:
df1=pd.DataFrame(irr_aoi)  

In [ ]:
df1.to_csv('E:\IITB Sem 2\EE770\Assignment 4\leh_aoi.csv') #Change

In [ ]:
df1=pd.read_csv('E:\IITB Sem 2\EE770\Assignment 4\leh_aoi.csv') # bcoz irr_aoi has timestamp #Change

In [ ]:
df_aoi=df1["aoi"]

# call tmy files

In [ ]:
df=pd.read_csv('E:\IITB Sem 2\EE770\Assignment 4\EE770_Leh.csv') #Change

In [ ]:
#df

In [ ]:
#df["RH(%)"]

# calculate gnd_diffuse

In [ ]:
df_ghi=df["Gh(W/m2)"]
gnd_diffuse=pvlib.irradiance.get_ground_diffuse(surface_tilt=location_tilt, ghi=df_ghi, albedo=0.25, surface_type=None)

In [ ]:
#df_ghi

# calculate sky_diffuse

In [ ]:
df_dhi=df["Gdhi(W/m2)"]
df_dni=df["Gdni(W/m2)"]

In [ ]:
s_diffuse= pvlib.irradiance.get_sky_diffuse(surface_tilt=location_tilt, surface_azimuth=location_azimuth, solar_zenith=solarpos.zenith, solar_azimuth=solarpos.azimuth, dni=df_dni, ghi=df_ghi, dhi=df_dhi, dni_extra=None, airmass=None)

In [ ]:
irr_poa=pvlib.irradiance.poa_components(aoi=df_aoi, dni=df_dni, poa_sky_diffuse=s_diffuse, poa_ground_diffuse=gnd_diffuse)

In [ ]:
#print(irr_poa)

In [ ]:
df2=pd.DataFrame(irr_poa)

In [ ]:
df2.to_csv('E:\IITB Sem 2\EE770\Assignment 4\leh_PVlib_poa.csv') #Change

# Sandia model

# Calculate Eb

In [ ]:
import math
import numpy as np

In [ ]:
solarzen=solarpos.zenith.tolist()
solarazi=solarpos.azimuth.tolist()

In [ ]:
rlocation_tilt=math.radians(location_tilt)
rlocation_azimuth=math.radians(location_azimuth)
rsolarzen=np.radians(solarzen)
rsolarazi=np.radians(solarazi)

In [ ]:
rsolarzen1=rsolarzen.tolist()
rsolarazi1=rsolarazi.tolist()

In [ ]:
from math import cos
from math import sin
from math import acos

In [ ]:
irr_aoi1=[]
for i in range(8760):
    irraoi=math.acos((cos(rsolarzen[i])*cos(rlocation_tilt))+(sin(rsolarzen[i])*sin(rlocation_tilt)*cos(rsolarazi[i]-rlocation_azimuth)))
    irr_aoi1.append(irraoi)

In [ ]:
df_dhi1=df_dhi.tolist() 
df_dni1=df_dni.tolist()
df_ghi1=df_ghi.tolist()

In [ ]:
Ebeam=[]
for i in range(8760):
    Eb=df_dni1[i]*cos(irr_aoi1[i])
    Ebeam.append(Eb)

# Calculate poa gnd_reflect

In [ ]:
Egnd=[]
for i in range(8760):
    Eg=df_ghi1[i]*0.25*((1-cos(rlocation_tilt))/2)
    Egnd.append(Eg)

# Calculate poa sk

In [ ]:
Ediff=[]
for i in range(8760):
    Ed=df_dhi1[i]*((1+cos(rlocation_tilt))/2)
    Ediff.append(Ed)

# calculate poa

In [ ]:
irr_poa1=[]
for i in range(8760):
    poa1=Ebeam[i]+Egnd[i]+Ediff[i]
    irr_poa1.append(poa1)
    

In [ ]:
df3=pd.DataFrame(irr_poa1)

In [ ]:
df3.to_csv('E:\IITB Sem 2\EE770\Assignment 4\leh_sandia_poa.csv') #Change

# calculate hourly temperature

In [ ]:
df_amb=df["amb_temp(C.)"]

In [ ]:
df_ws=df["ws(m/s)"]

In [ ]:
module_temp=pvlib.temperature.sapm_module(poa_global=df2["poa_global"], temp_air=df_amb, wind_speed=df_ws, a=-3.56, b=-0.075)

In [ ]:
#df_ws1=df_ws.tolist()
#f_amb1=df_amb.tolist()

In [ ]:
#module_temp1=[]
#for i in range(8760):
    #mod=irr_poa1[i]*math.exp(-3.56+(-0.0750*df_ws1[i]))+df_amb1[i]
    #module_temp1.append(mod)

In [ ]:
df=pd.DataFrame(module_temp)
df.to_csv('E:\IITB Sem 2\EE770\Assignment 4\leh_temp.csv') #Change

# Q2 box plot

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.boxplot(module_temp)
plt.title('Leh Module Temp') #Change
plt.ylabel('Temp in C')
plt.show()

In [ ]:
plt.boxplot(df2["poa_global"])
plt.title('Leh Plane of array Irradiance') #Change
plt.ylabel('Irr in W/m2')
plt.show()

In [ ]:
plt.boxplot(df_amb)
plt.title('Leh Ambient Temp') #Change
plt.ylabel('Temp in C')
plt.show()

In [ ]:
plt.boxplot(df_ws)
plt.title('Leh Wind speed') #Change
plt.ylabel('Speen in m/s')
plt.show()

# Q3

In [ ]:
from scipy.optimize import fsolve


In [ ]:
Rs_STC= 0.21721509749730586
Rsh_STC= 1104.8948729303456
Iph_STC= 8.841737886118327
Isat_STC= 2.1922584839244864e-07
eta= 1.3974467720580908
Vt_STC= 0.02571543749999999
Ncell=72
Voc_STC= 45.300000000011885
Isc_STC= 8.839999758809823
Pmax_STC= 300.1199710523156
FF_STC= 0.7494530661370086
alpha=0.005304 #alpha(A/degC)
Kb = 1.38 * (10**-23)
q = 1.6 * (10**-19)

In [ ]:
G_STC=1000
T_STC=298 #radians
Ksc=0.997951 # considering NOCT values
Eg=1.14
T1=module_temp.tolist()
T=[x + 273 for x in module_temp] 
G=df2["poa_global"].tolist()

In [ ]:
Voc_T=[]
Isc_T=[]
Iph_T=[]
Isat_T=[]
Vt_T=[]
Rsh_T=[]
for i in range(8760):
    if G[i]==0:
        Voc_T.append(0)
        Isc_T.append(0)
        Iph_T.append(0)
        Isat_T.append(0)
        Vt_T.append(0)
        Rsh_T.append(0)
    else:
        Vt = (Kb * T[i]) / q
        Isc_GT= ((G[i]/G_STC)**Ksc) * (Isc_STC+alpha*(T[i]-T_STC))
        Rs_GT=Rs_STC
        Rsh_GT=(G_STC/G[i])*Rsh_STC
        Iph_GT= Isc_GT*(1+(Rs_GT/Rsh_GT))
        Isat_GT= Isat_STC * ((T[i]/T_STC)**3) * np.exp(((q*Eg)/(eta*Kb))*((1/T_STC)-(1/T[i])))
        def V_oc(x):
            Voc=x[0]
            F=np.zeros(np.size(x))
            F[0]= Iph_GT-Isat_GT*(np.exp(Voc/(eta*Ncell*Vt)))- (Voc/Rsh_GT)
            return F

        x=fsolve(V_oc,100)
        Voc_T.extend(x)
        Isc_T.append(Isc_GT)
        Iph_T.append(Iph_GT)
        Isat_T.append(Isat_GT)
        Vt_T.append(Vt)
        Rsh_T.append(Rsh_GT)


In [ ]:
rows=8760
V2= [[0 for k in range(0)] for j in range(rows)] #Creating array inside an array 
I2= [[0 for k in range(0)] for j in range(rows)] 

In [ ]:
for i in range(8760):
    def I_sc(x):
        I=x[0]
        F=np.zeros(np.size(x))
        F[0]= -I + Iph_T[i] - Isat_T[i]*(np.exp((V1+I*Rs_GT) / (eta*Ncell*Vt_T[i]))-1) - (V1+I*Rs_GT)/Rsh_T[i]
        return F
    Vx=np.linspace(0,Voc_T[i],num=256)
    for V1 in Vx:
        if G[i]==0:
            I2[i].append(0)
            V2[i].append(0)
        else:
            x = fsolve(I_sc,Iph_STC)
            I2[i].extend(x)
            V2[i].append(V1)

In [ ]:
len(V2[1])

In [ ]:
len(I2[1])

In [ ]:
P2=[]
Pmax2=[]
for i in range(8760):
    P2=np.multiply(V2[i],I2[i])
    Pmax2.append(np.amax(P2))

In [ ]:
len(Pmax2)

In [ ]:
df_Pmax=pd.DataFrame(Pmax2)

In [ ]:
df_Pmax.to_csv('E:\IITB Sem 2\EE770\Assignment 4\leh_Power.csv') #Change

In [ ]:
np.sum(Pmax2)

In [ ]:
df_poa1=pd.read_csv('E:\IITB Sem 2\EE770\Assignment 4\jaipur_PVlib_poa.csv')

In [ ]:
df_poa2=pd.read_csv('E:\IITB Sem 2\EE770\Assignment 4\jaipur_sandia_poa.csv')

In [ ]:
#df_poa1["poa_global"]

In [ ]:
plt.plot(df_poa1["poa_global"])
plt.title('Jaipur Hourly POA (pvlib)') #Change
plt.xlabel('Time in Hours')
plt.ylabel('Irr in W/m2')
plt.xticks(np.arange(0,10000,1000))
plt.axis([0, 9000, 0, 1200])
plt.show()

In [ ]:
plt.plot(df_poa2,color='tab:blue')
plt.title('Jaipur Hourly POA (sandia)') #Change
plt.xlabel('Time in Hours')
plt.ylabel('Irr in W/m2')
plt.xticks(np.arange(0,10000,1000))
plt.axis([0, 9000, 0, 1200])
plt.show()

In [ ]:
df_temp=pd.read_csv('E:\IITB Sem 2\EE770\Assignment 4\jaipur_temp.csv')

In [ ]:
plt.plot(df_temp,color='tab:blue')
plt.title('Jaipur Hourly Module Temperature') #Change
plt.ylabel('Temp in C')
plt.xticks(np.arange(0,10000,1000))
plt.axis([0, 9000,0, 70])
plt.show()

In [ ]:
df_power=pd.read_csv('E:\IITB Sem 2\EE770\Assignment 4\leh_Power.csv')

In [ ]:
plt.plot(df_power,color='tab:blue')
plt.title('Leh Annual Energy generation ') #Change
plt.ylabel('P in watts')
plt.xlabel('Time in Hours')
plt.xticks(np.arange(0,10000,1000))
plt.axis([0, 9000,0, 350])
plt.show()